# Slack to Jira Workflow Agent

This notebook demonstrates how to build an intelligent workflow agent that creates Jira stories based on Slack discussions and links them to existing epics.

### Agent Example:

This notebook will walkthrough how to build a system that can perform the following task via an agent built with Llama Stack:

- *"Analyze a Slack discussion thread, extract key information about a technical issue or feature request, create a well-structured Jira story with appropriate fields, and link it to a specified epic."*

### MCP Tools:

#### Slack MCP Server
We will be using the [Slack MCP Server](https://github.com/modelcontextprotocol/servers/tree/main/src/slack) to retrieve discussion content from Slack channels and threads.

#### Jira MCP Server
We will also be using the [Jira MCP Server](https://github.com/sooperset/mcp-atlassian) to create and manage Jira issues, including linking stories to epics.

## Pre-Requisites

Before starting this notebook, ensure that you have:
- Followed the instructions in the [Setup Guide](./Level0_getting_started_with_Llama_Stack.ipynb) notebook.
- Access to a Slack workspace with the Slack MCP server configured
- Access to a Jira instance with the Jira MCP server configured
- Appropriate permissions to create issues in Jira and read messages from Slack

## Setting Up this Notebook
We will initialize our environment as described in detail in our ["Getting Started" notebook](./Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [1]:
# for accessing the environment variables
import os
from dotenv import load_dotenv
load_dotenv()

# for communication with Llama Stack
from llama_stack_client import LlamaStackClient
from llama_stack_client import Agent
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from llama_stack_client.lib.agents.event_logger import EventLogger

# pretty print of the results returned from the model/agent
from termcolor import cprint
import sys
import json
sys.path.append('..')  
from src.utils import step_printer

base_url = os.getenv("REMOTE_BASE_URL")

# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}

client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)

print(f"Connected to Llama Stack server")

# model_id for the model you wish to use that is configured with the Llama Stack server
model_id = "granite32-8b"

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = 512

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

ModuleNotFoundError: No module named 'fire'

## Validate tools are available in our Llama Stack instance

When an instance of Llama Stack is redeployed, it may be the case that the tools will need to be re-registered. Also if a tool is already registered with a Llama Stack instance, trying to register another one with the same `toolgroup_id` will throw you an error.

For this reason, it is recommended to validate your tools and toolgroups. The following code will check that both the `mcp::slack` and `mcp::jira` tools are correctly registered, and if not it will attempt to register them using their specific endpoints.

In [ ]:
slack_mcp_url = os.getenv("REMOTE_SLACK_MCP_URL")
jira_mcp_url = os.getenv("REMOTE_JIRA_MCP_URL")

registered_tools = client.tools.list()
registered_toolgroups = [t.toolgroup_id for t in registered_tools]

if "mcp::slack" not in registered_toolgroups:
    client.toolgroups.register(
        toolgroup_id="mcp::slack",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":slack_mcp_url},
   )

if "mcp::jira" not in registered_toolgroups:
    client.toolgroups.register(
        toolgroup_id="mcp::jira",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":jira_mcp_url},
    )

print(f"Your Llama Stack server is registered with the following tool groups @ {set(registered_toolgroups)} \n")

## Defining our Agent - Slack to Jira Workflow

In [ ]:
model_prompt = """You are a helpful assistant specialized in analyzing Slack discussions and creating well-structured Jira stories.

When analyzing Slack discussions, you should:
1. Extract the main technical issue, bug report, or feature request from the conversation
2. Identify key details like:
   - Problem description
   - Steps to reproduce (if mentioned)
   - Expected vs actual results (if discussed)
   - Error messages or symptoms
   - Proposed solutions or workarounds
3. Create a clear, concise summary for the Jira story title
4. Write a detailed description that includes context from the Slack discussion
5. Link the story to the appropriate epic when specified

Always be thorough in your analysis and create professional, well-formatted Jira stories that capture the essence of the Slack discussion.
"""

### Analyze Slack Discussion and Create Jira Story

Now we'll create an agent that can analyze a specific Slack discussion thread and create a corresponding Jira story. The agent will:

1. Retrieve the Slack thread messages
2. Analyze the discussion to extract key information
3. Create a well-structured Jira story
4. Link it to the specified epic

Let's start by analyzing the Slack discussion from the provided URL.

In [ ]:
# Configuration for the specific Slack discussion and Jira epic
SLACK_CHANNEL_ID = "C08UPFE3VA5"
SLACK_THREAD_TS = "1748436131.235159"
JIRA_EPIC_KEY = "ILAICOMUTY-6"
JIRA_PROJECT_KEY = "ILAICOMUTY"

# Create agent with Slack and Jira tools
agent = ReActAgent(
    client=client,
    model=model_id,
    tools=["mcp::slack", "mcp::jira"],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
    sampling_params={"max_tokens": 1024},
)

# Define the workflow prompt
workflow_prompt = f"""
Please perform the following workflow:

1. First, retrieve the Slack thread messages from channel {SLACK_CHANNEL_ID} with thread timestamp {SLACK_THREAD_TS}
2. Analyze the discussion to understand the main technical issue or topic being discussed
3. Get details about the epic {JIRA_EPIC_KEY} to understand the context
4. Create a new Jira story in project {JIRA_PROJECT_KEY} with:
   - A clear, descriptive summary based on the Slack discussion
   - A detailed description that includes:
     * Summary of the Slack discussion
     * Key technical details mentioned
     * Any error messages or symptoms described
     * Steps to reproduce (if mentioned)
     * Expected vs actual results (if discussed)
   - Link it to epic {JIRA_EPIC_KEY} using the parent field
   - Set appropriate issue type as "Story"

Make sure to create a professional, well-structured Jira story that captures the essence of the Slack discussion.
"""

session_id = agent.create_session("slack-to-jira-workflow")

print("\n" + "="*80)
cprint(f"Starting Slack to Jira Workflow", "blue")
print("="*80)

response = agent.create_turn(
    messages=[
        {
            "role": "user",
            "content": workflow_prompt,
        }
    ],
    session_id=session_id,
    stream=stream
)

if stream:
    for log in EventLogger().log(response):
        log.print()
else:
    step_printer(response.steps)

### Output Analysis

The workflow agent successfully performed the following steps:

1. **Retrieved Slack Thread Messages**: The agent used the Slack MCP server to fetch the complete thread discussion, including all replies and context.

2. **Analyzed the Discussion**: The agent identified the main technical issue (Python ModuleNotFoundError with requests library) and extracted key details from the conversation.

3. **Retrieved Epic Context**: The agent fetched details about the target epic to understand the project context and ensure proper linking.

4. **Created Jira Story**: The agent created a well-structured Jira story with:
   - Clear, descriptive summary
   - Detailed description including Slack discussion context
   - Proper linking to the parent epic
   - Appropriate issue type and project assignment

This demonstrates how MCP tools can be orchestrated to create seamless integrations between different platforms, enabling automated workflow processes that preserve context and maintain traceability.

## Key Takeaways

This notebook demonstrated how to build a sophisticated workflow agent that bridges Slack and Jira using MCP tools with Llama Stack. Key achievements include:

1. **Seamless Integration**: Connected two different platforms (Slack and Jira) through a unified agent interface
2. **Context Preservation**: Maintained discussion context when creating Jira stories from Slack threads
3. **Intelligent Analysis**: Used LLM capabilities to extract meaningful information from unstructured Slack conversations
4. **Automated Workflow**: Created an end-to-end process that reduces manual effort in issue tracking
5. **Flexible Architecture**: Built a reusable framework that can be adapted for different use cases

This approach can be extended to support more complex workflows, such as:
- Automatic priority and severity detection
- Multi-platform issue synchronization
- Intelligent routing based on discussion content
- Integration with additional tools like GitHub, ServiceNow, or custom APIs

The combination of MCP tools and Llama Stack provides a powerful foundation for building intelligent automation workflows that enhance team productivity and ensure important discussions don't get lost in chat platforms.